In [1]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [28]:
counters = sqlContext.read.format("org.apache.spark.sql.cassandra").load(table="counters", keyspace="dice")

In [3]:
counters.printSchema()

root
 |-- id: string (nullable = true)
 |-- modified: timestamp (nullable = true)
 |-- contentname: string (nullable = true)
 |-- crsid: string (nullable = true)
 |-- description: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- stevci_cestaopis: string (nullable = true)
 |-- stevci_gap: float (nullable = true)
 |-- stevci_hit: integer (nullable = true)
 |-- stevci_lokacijaopis: string (nullable = true)
 |-- stevci_pasopis: string (nullable = true)
 |-- stevci_smeropis: string (nullable = true)
 |-- stevci_stat: integer (nullable = true)
 |-- stevci_statopis: string (nullable = true)
 |-- stevci_stev: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- x: float (nullable = true)
 |-- x_wgs: float (nullable = true)
 |-- y: float (nullable = true)
 |-- y_wgs: float (nullable = true)



In [29]:
counters.groupBy('stevci_stat', 'stevci_statopis').count().show()

+-----------+--------------------+-----+
|stevci_stat|     stevci_statopis|count|
+-----------+--------------------+-----+
|          1|     Normalen promet| 3953|
|          5|Gost promet z zas...|   27|
|          4|         Gost promet|  585|
|          3|      Zgoščen promet|  472|
|          2|      Povečan promet| 1523|
|          6|          Ni prometa| 1151|
+-----------+--------------------+-----+



In [30]:
counters = counters[counters.stevci_stat =b= 4].select('id', 'stevci_lokacijaopis', 'stevci_cestaopis', 'stevci_smeropis', 'stevci_pasopis', 'x_wgs', 'y_wgs', 'modified')

In [31]:
counters.groupBy('id','stevci_lokacijaopis', 'stevci_cestaopis', 'stevci_smeropis').count().show()

+----+--------------------+----------------+--------------------+-----+
|  id| stevci_lokacijaopis|stevci_cestaopis|     stevci_smeropis|count|
+----+--------------------+----------------+--------------------+-----+
|0178|LJ (južna obvoznica)|           AC-A1|Peruzzijeva - Bar...|  100|
|0832|     Kamna Gorica AC|           AC-A2|    Šentvid - Koseze|   51|
|0120|           Šentjakob|          G2-108|  Črnuče - Šentjakob|    1|
|0747|          LJ Šmartno|          R3-644|Sneberje - Obvoznica|    5|
|0120|           Šentjakob|          G2-108|  Šentjakob - Črnuče|    5|
|0180| LJ (vzh. obvoznica)|           AC-A1|  Malence - Litijska|   75|
|0178|LJ (južna obvoznica)|           AC-A1|Barjanska - Peruz...|   18|
|0174|    LJ (S obvoznica)|           HC-H3|LJ Savlje - LJ Du...|   76|
|0725|LJ (Celovška Drav...|            G1-8| Šentvid - obvoznica|   51|
|0179|LJ (SV obvoznica HC)|           HC-H3|Šmartinska - Zado...|   65|
|0892|Priključek Nove J...|           HC-H3|  Vstop na HC iz BTC

In [32]:
sorted(counters.select('modified', 'stevci_smeropis').collect(), key=lambda x: x['modified'])

[Row(modified=datetime.datetime(2017, 4, 7, 16, 0, 6, 197000), stevci_smeropis=u'Peruzzijeva - Barjanska'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 0, 6, 197000), stevci_smeropis=u'\u0160martinska - Zadobrova'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 0, 6, 197000), stevci_smeropis=u'Vstop na HC iz BTC'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 0, 6, 197000), stevci_smeropis=u'Malence - Litijska'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 39, 28, 225000), stevci_smeropis=u'Peruzzijeva - Barjanska'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 39, 28, 225000), stevci_smeropis=u'Celov\u0161ka - Koseze'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 39, 28, 225000), stevci_smeropis=u'\u0160martinska - Zadobrova'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 39, 28, 225000), stevci_smeropis=u'Vstop na HC iz BTC'),
 Row(modified=datetime.datetime(2017, 4, 7, 16, 59, 10, 517000), stevci_smeropis=u'Peruzzijeva - Barjanska'),
 Row(modified=datetime.dat

In [26]:
counters.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="counters_ht", keyspace="dice_results").save()